In [1]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
from MO_agent import *

#### making the environment

In [2]:
e=tiger_POMDP_env(read_config=True,config_address='./tiger.json',parameters=None)

{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.3, 'false_observation_get_r_while_l': 0.3, 'reward_listen': -0.5, 'reward_low_incorrect': -1, 'reward_high_incorrect': -2, 'reward_low_correct': 1, 'reward_high_correct': 2, 'discount_factor': 0.9, 'initial_wealth': 0}


#### Setting simulation parameters

In [3]:
e.reset()
initial_observation=[e.current_state]
observation=initial_observation
state=e.current_state
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))


In [12]:
# parameters
exp_vorfaktoren=[-0.5,0.1]
partitioning_chunk_number=100
initial_theta=[0.5,0.5]
initial_observation=0
planning_depth=3
maximum_depth=planning_depth

internal_state=initial_theta*len(exp_vorfaktoren)
internal_state.extend([initial_observation])

#### Simulation

In [15]:
ag=Multi_Variate_agent(environment=e, planning_depth=planning_depth, partitioning_chunk_number=partitioning_chunk_number,agent_mode='naive')
ag.reset()
ag.pre_planning(exp_vorfaktoren, initial_theta=initial_theta,initial_observation=initial_observation)
ag.value_iteration()
for t in range(maximum_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action=ag.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_state)
    print('')
    print('============================')
    

t= 0
------
state: 0 tiger_right
last_observation: 1
-------------------------------

current internal state: [0.5, 0.5, 0.5, 0.5, 0]

             action: listen
             **
reward: -0.5 new observation: 0

new internal state: [0.7, 0.3, 0.7, 0.3, 0]

t= 1
------
state: 0 tiger_right
last_observation: 0
-------------------------------

current internal state: [0.7, 0.3, 0.7, 0.3, 0]

             action: open_left_low
             **
reward: 1.0 new observation: 1

new internal state: [0.5, 0.5, 0.5, 0.5, 1]

t= 2
------
state: 0 tiger_right
last_observation: 1
-------------------------------

current internal state: [0.5, 0.5, 0.5, 0.5, 1]

             action: open_right_low
             **
reward: -1.0 new observation: 0

new internal state: [0.5, 0.5, 0.5, 0.5, 0]

